## Latent Dirichlet Allocation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

# http://qwone.com/~jason/20Newsgroups/
dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    categories=['sci.electronics', 'sci.crypt', 'sci.med', 'sci.space']
    # categories=['rec.sport.baseball', 'sci.crypt', 'comp.graphics', 'talk.politics.guns']
)
documents = dataset.data
labels = dataset.target

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(
    max_df=0.95, 
    min_df=2, 
    max_features=100, 
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(documents)
feature_names = tf_vectorizer.get_feature_names()

# Run LDA
model = LDA(
    n_topics=4, 
    max_iter=5, 
    learning_method='batch', 
    learning_offset=50.,
    random_state=0
).fit(tf)

# Show keywords
no_top_words = 10
for topic_idx, topic in enumerate(model.components_):
    print "Topic %d:" % (topic_idx)
    print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups

categories = ['rec.sport.hockey', 'talk.religion.misc', 'soc.religion.christian']

# training navie bayes
trained_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='train',
    categories=categories
)
X0 = trained_dataset.data
y0 = trained_dataset.target
h_pipeline = Pipeline([
    ('vec', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB(alpha=.1)),
])
%timeit h_pipeline.fit(X0, y0)

# prediction
test_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='test',
    categories=categories
)
X1 = test_dataset.data
y1 = test_dataset.target
test_label_names = test_dataset.target_names
h_pipeline.score(X1, y1)
predicted_y1 = h_pipeline.predict(X1)
print(metrics.classification_report(
    y1, 
    predicted_y1, 
    target_names=test_label_names
))

1 loop, best of 3: 442 ms per loop
                        precision    recall  f1-score   support

      rec.sport.hockey       0.93      0.97      0.95       399
soc.religion.christian       0.69      0.95      0.80       398
    talk.religion.misc       0.93      0.33      0.49       251

           avg / total       0.84      0.81      0.78      1048



In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

categories = ['rec.sport.hockey', 'talk.religion.misc', 'soc.religion.christian']

# prepare lda model
all_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    categories=categories
)
X = all_dataset.data
tf_vectorizer = CountVectorizer(
    max_df=0.95, 
    min_df=2, 
    max_features=1000, 
    stop_words='english'
)
tf = tf_vectorizer.fit_transform(X)
lda = LDA(
    n_topics=3, 
    max_iter=5, 
    learning_method='batch', 
    learning_offset=50.,
    random_state=0
).fit(tf)
no_top_words = 10
feature_names = tf_vectorizer.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print "Topic %d:" % (topic_idx)
    print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])

# training navie bayes
trained_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='train',
    categories=categories
)
X0 = trained_dataset.data
y0 = trained_dataset.target
h_pipeline = Pipeline([
    ('vec', TfidfVectorizer(stop_words='english')),
    ('lda', lda),
    ('clf', MultinomialNB(alpha=.1)),
])
%timeit h_pipeline.fit(X0, y0)

# prediction
test_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='test',
    categories=categories
)
X1 = test_dataset.data
y1 = test_dataset.target
test_label_names = test_dataset.target_names
h_pipeline.score(X1, y1)
predicted_y1 = h_pipeline.predict(X1)
print(metrics.classification_report(
    y1, 
    predicted_y1, 
    target_names=test_label_names
))

Topic 0:
god people jesus think does know don say just believe
Topic 1:
team game hockey season games year play nhl players like
Topic 2:
10 25 55 11 12 16 14 15 18 13
1 loop, best of 3: 2.74 s per loop
                        precision    recall  f1-score   support

      rec.sport.hockey       0.75      0.32      0.45       399
soc.religion.christian       0.43      0.95      0.59       398
    talk.religion.misc       0.00      0.00      0.00       251

           avg / total       0.45      0.48      0.39      1048



In [3]:
# https://github.com/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

pyLDAvis.enable_notebook()

newsgroups = fetch_20newsgroups(
    remove=('headers', 'footers', 'quotes')
)
docs_raw = newsgroups.data
print(len(docs_raw))
tf_vectorizer = CountVectorizer(
    strip_accents = 'unicode',
    stop_words = 'english',
    lowercase = True,
    token_pattern = r'\b[a-zA-Z]{3,}\b',
    max_df = 0.5, 
    min_df = 10
)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)
# for TF DTM
lda_tf = LDA(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LDA(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

11314
(11314, 9144)
(11314, 9144)


/usr/local/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
15     10.823712        1       1  0.092144 -0.139924
12      8.300551        1       2  0.202586  0.013026
4       8.237988        1       3  0.227601  0.049152
6       7.815383        1       4  0.071434 -0.047499
0       7.529769        1       5  0.152673  0.006487
14      6.829115        1       6  0.097501  0.107442
9       6.519449        1       7 -0.065267 -0.105437
13      6.480666        1       8  0.120288 -0.083854
5       6.442044        1       9  0.191532  0.091312
8       5.365540        1      10  0.144040  0.043591
2       4.779361        1      11 -0.164598 -0.219685
1       3.899585        1      12  0.164470  0.025632
18      3.288486        1      13  0.010929 -0.146494
16      2.745557        1      14 -0.153248  0.252591
10      2.693579        1      15 -0.183087  0.008268
7       2.446077        1      16 -0.136747 -0.065023
19      2.278135        1      17 -0.201555  0.040887
3       1.845921        1      18 -0.199580 -0.161042
11      0.902157        1      19 -0.190450  0.121579
17      0.776925        1      20 -0.180665  0.208991, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
5016  Default  4394.000000         max  4394.000000  30.0000  30.0000
2653  Default  2391.000000         edu  2391.000000  29.0000  29.0000
3523  Default  2039.000000         god  2039.000000  28.0000  28.0000
1506  Default  1432.000000         com  1432.000000  27.0000  27.0000
3153  Default  1678.000000        file  1678.000000  26.0000  26.0000
2560  Default  1140.000000       drive  1140.000000  25.0000  25.0000
4895  Default  1183.000000        mail  1183.000000  24.0000  24.0000
8982  Default  1192.000000     windows  1192.000000  23.0000  23.0000
4543  Default  3503.000000        know  3503.000000  22.0000  22.0000
4769  Default   933.000000        list   933.000000  21.0000  21.0000
7677  Default  1263.000000       space  1263.000000  20.0000  20.0000
2509  Default  3876.000000         don  3876.000000  19.0000  19.0000
2077  Default  1232.000000        data  1232.000000  18.0000  18.0000
5924  Default  4066.000000      people  4066.000000  17.0000  17.0000
912   Default   649.000000        book   649.000000  16.0000  16.0000
627   Default  1321.000000   available  1321.000000  15.0000  15.0000
7329  Default   841.000000        send   841.000000  14.0000  14.0000
7629  Default   946.000000    software   946.000000  13.0000  13.0000
6362  Default  1377.000000     program  1377.000000  12.0000  12.0000
3986  Default   682.000000       image   682.000000  11.0000  11.0000
4468  Default  3743.000000        just  3743.000000  10.0000  10.0000
3156  Default   831.000000       files   831.000000   9.0000   9.0000
4396  Default   862.000000       jesus   862.000000   8.0000   8.0000
8278  Default  2997.000000       think  2997.000000   7.0000   7.0000
3551  Default  1211.000000  government  1211.000000   6.0000   6.0000
2517  Default   678.000000         dos   678.000000   5.0000   5.0000
8252  Default  1249.000000      thanks  1249.000000   4.0000   4.0000
1161  Default   812.000000        card   812.000000   3.0000   3.0000
8167  Default   800.000000        team   800.000000   2.0000   2.0000
7250  Default   620.000000        scsi   620.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
8050  Topic20   100.947391   surrender   101.893082   4.8483  -4.1431
5547  Topic20    98.664750         nyi    99.610438   4.8480  -4.1660
5549  Topic20    99.663611         nyr   100.645122   4.8478  -4.1559
1614  Topic20    94.986822    compound    95.932834   4.8477  -4.2040
7552  Topic20    85.149172  skepticism    86.094860   4.8465  -4.3133
7860  Topic20   110.355515         stl   111.598819   4.8464  -4.0540
2650  Topic20    77.915052         edm 

## Spam Detection 

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/Spambase
# https://github.com/zhangruiskyline/NLP_DeepLearning#project-1-spam-detector
# picture flow explain lda ~
# https://www.cs.cmu.edu/~./enron/ 
# another email dataset

